In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
 #Load the libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import re
import string
import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from bs4 import BeautifulSoup

from keras.preprocessing.sequence import pad_sequences

import os
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#importing the training data
train_data=pd.read_csv('/content/drive/MyDrive/Reviews.csv')
print(train_data.shape)
train_data.head(10)

(568454, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Review
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...
6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,5,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...
7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,5,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...
8,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,5,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...
9,10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,5,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...


In [ ]:
df_data = pd.DataFrame(train_data.Score, columns = ['Score'])
df_data['review']= train_data.Review
df_data.head(10)

,Score,review
0,5,I have bought several of the Vitality canned d...
1,1,Product arrived labeled as Jumbo Salted Peanut...
2,4,This is a confection that has been around a fe...
3,2,If you are looking for the secret ingredient i...
4,5,Great taffy at a great price. There was a wid...
5,4,I got a wild hair for taffy and ordered this f...
6,5,This saltwater taffy had great flavors and was...
7,5,This taffy is so good. It is very soft and ch...
8,5,Right now I'm mostly just sprouting this so my...
9,5,This is a very healthy dog food. Good for thei...


In [ ]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

df_data['review']=df_data['review'].apply(strip_html)

In [ ]:
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
df_data['review']= df_data['review'].apply(remove_special_characters)
df_data['review'].head(10)

0    I have bought several of the Vitality canned d...
1    Product arrived labeled as Jumbo Salted Peanut...
2    This is a confection that has been around a fe...
3    If you are looking for the secret ingredient i...
4    Great taffy at a great price  There was a wide...
5    I got a wild hair for taffy and ordered this f...
6    This saltwater taffy had great flavors and was...
7    This taffy is so good  It is very soft and che...
8    Right now Im mostly just sprouting this so my ...
9    This is a very healthy dog food Good for their...
Name: review, dtype: object

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
df_data['review'] = df_data['review'].apply(lambda x: tokenizer.tokenize(x.lower()))
df_data['review'].head(10)

0    [i, have, bought, several, of, the, vitality, ...
1    [product, arrived, labeled, as, jumbo, salted,...
2    [this, is, a, confection, that, has, been, aro...
3    [if, you, are, looking, for, the, secret, ingr...
4    [great, taffy, at, a, great, price, there, was...
5    [i, got, a, wild, hair, for, taffy, and, order...
6    [this, saltwater, taffy, had, great, flavors, ...
7    [this, taffy, is, so, good, it, is, very, soft...
8    [right, now, im, mostly, just, sprouting, this...
9    [this, is, a, very, healthy, dog, food, good, ...
Name: review, dtype: object

In [ ]:
def remove_stopwords (text):
  words = [w for w in text if w not in stopwords.words('english')]
  return words

df_data['review'] = df_data['review'].apply(lambda x : remove_stopwords(x))
df_data['review'].head(10)

0    [bought, several, vitality, canned, dog, food,...
1    [product, arrived, labeled, jumbo, salted, pea...
2    [confection, around, centuries, light, pillowy...
3    [looking, secret, ingredient, robitussin, beli...
4    [great, taffy, great, price, wide, assortment,...
5    [got, wild, hair, taffy, ordered, five, pound,...
6    [saltwater, taffy, great, flavors, soft, chewy...
7    [taffy, good, soft, chewy, flavors, amazing, w...
8    [right, im, mostly, sprouting, cats, eat, gras...
9    [healthy, dog, food, good, digestion, also, go...
Name: review, dtype: object

In [ ]:
df = pd.DataFrame(df_data.review, columns = ['review'])
#df['abstract']= df_data['abstract']
df['review'].head(10)

0    [bought, several, vitality, canned, dog, food,...
1    [product, arrived, labeled, jumbo, salted, pea...
2    [confection, around, centuries, light, pillowy...
3    [looking, secret, ingredient, robitussin, beli...
4    [great, taffy, great, price, wide, assortment,...
5    [got, wild, hair, taffy, ordered, five, pound,...
6    [saltwater, taffy, great, flavors, soft, chewy...
7    [taffy, good, soft, chewy, flavors, amazing, w...
8    [right, im, mostly, sprouting, cats, eat, gras...
9    [healthy, dog, food, good, digestion, also, go...
Name: review, dtype: object

In [ ]:
stemmer = PorterStemmer()

def word_stemmer (text):
  stem_text = [stemmer.stem(i) for i in text]
  return stem_text

df['review'] = df['review'].apply(lambda x: word_stemmer(x))
df.head(10)

,review
0,"[bought, sever, vital, can, dog, food, product..."
1,"[product, arriv, label, jumbo, salt, peanutsth..."
2,"[confect, around, centuri, light, pillowi, cit..."
3,"[look, secret, ingredi, robitussin, believ, fo..."
4,"[great, taffi, great, price, wide, assort, yum..."
5,"[got, wild, hair, taffi, order, five, pound, b..."
6,"[saltwat, taffi, great, flavor, soft, chewi, c..."
7,"[taffi, good, soft, chewi, flavor, amaz, would..."
8,"[right, im, mostli, sprout, cat, eat, grass, l..."
9,"[healthi, dog, food, good, digest, also, good,..."


In [ ]:
df['score'] = df_data['Score']
df.head(10)

,review,score
0,"[bought, sever, vital, can, dog, food, product...",5
1,"[product, arriv, label, jumbo, salt, peanutsth...",1
2,"[confect, around, centuri, light, pillowi, cit...",4
3,"[look, secret, ingredi, robitussin, believ, fo...",2
4,"[great, taffi, great, price, wide, assort, yum...",5
5,"[got, wild, hair, taffi, order, five, pound, b...",4
6,"[saltwat, taffi, great, flavor, soft, chewi, c...",5
7,"[taffi, good, soft, chewi, flavor, amaz, would...",5
8,"[right, im, mostli, sprout, cat, eat, grass, l...",5
9,"[healthi, dog, food, good, digest, also, good,...",5
